In [1]:
import cvxpy as cp
import numpy as np
import math

np.set_printoptions(suppress=True, precision=3)

In [13]:
#CASE9    Power flow data for 9 bus, 3 generator case.

baseMVA = 100

# Bus data: bus_i, type, Pd, Qd, Gs, Bs, area, Vm, Va, baseKV, zone, Vmax, Vmin, Pg, Qg, Qmax, Qmin, Vg, mBase, status, Pmax, Pmin, startup, shutdown, n, c(n-1), ..., c0
# Index   :     0,    1,  2,  3,  4,  5,    6,  7,  8,      9,   10,   11,   12, 13, 14,   15,   16, 17,    18,     19,   20,   21,      22,       23,24,     25,  26, 27
bus_data = np.array([
    [1, 3, 0, 0, 0, 0, 1, 1, 0, 345, 1, 1.1, 0.9, 0, 0, 300, -300, 1, 100, 1, 250, 10, 1500, 0, 3, 0.11, 5, 150],
    [2, 2, 0, 0, 0, 0, 1, 1, 0, 345, 1, 1.1, 0.9, 163, 0, 300, -300, 1, 100, 1, 300, 10, 2000, 0, 3, 0.085,1.2, 600],
    [3, 2, 0, 0, 0, 0, 1, 1, 0, 345, 1, 1.1, 0.9, 85, 0, 300, -300, 1, 100, 1, 270, 10, 3000, 0, 3, 0.1225, 1, 335],
    [4, 1, 0, 0, 0, 0, 1, 1, 0, 345, 1, 1.1, 0.9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [5, 1, 90, 30, 0, 0, 1, 1, 0, 345, 1, 1.1, 0.9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [6, 1, 0, 0, 0, 0, 1, 1, 0, 345, 1, 1.1, 0.9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [7, 1, 100, 35, 0, 0, 1, 1, 0, 345, 1, 1.1, 0.9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [8, 1, 0, 0, 0, 0, 1, 1, 0, 345, 1, 1.1, 0.9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [9, 1, 125, 50, 0, 0, 1, 1, 0, 345, 1, 1.1, 0.9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
])


# Branch data: fbus, tbus, r, x, b, rateA, rateB, rateC
line_data = np.array([
    [1, 4, 0, 0.0576, 0, 250, 250, 250],
    [4, 5, 0.017, 0.092, 0.158, 250, 250, 250],
    [5, 6, 0.039, 0.17, 0.358, 150, 150, 150],
    [3, 6, 0, 0.0586, 0, 300, 300, 300],
    [6, 7, 0.0119, 0.1008, 0.209, 150, 150, 150],
    [7, 8, 0.0085, 0.072, 0.149, 250, 250, 250],
    [8, 2, 0, 0.0625, 0, 250, 250, 250],
    [8, 9, 0.032, 0.161, 0.306, 250, 250, 250],
    [9, 4, 0.01, 0.085, 0.176, 250, 250, 250]
])

In [14]:
# Assuming the number of nodes and lines
num_nodes = len(bus_data)  # Number of nodes
num_lines = len(line_data)  # Number of lines

# Define the Sending and receiving ends for each lines
sending_node = line_data[:,0].astype(int)
receiving_node = line_data[:,1].astype(int)

# Define the constants A+, A-, G_n, B_n, and p_d, q_d for demand
A_plus = np.zeros((num_lines,num_nodes)) # A+ matrix (num_nodes x num_lines)
for i in range(num_lines):
    A_plus[i,int(line_data[i,0]-1)] = 1
    A_plus[i,int(line_data[i,1]-1)] = -1

A_minus = np.zeros((num_lines,num_nodes)) # A- matrix (num_nodes x num_lines)
for i in range(num_lines):
    A_minus[i,int(line_data[i,0]-1)] = 0
    A_minus[i,int(line_data[i,1]-1)] = -1

A_plus = np.transpose(A_plus)
A_minus = np.transpose(A_minus)

G_n = bus_data[:,4]  # Nodal conductance
B_n = bus_data[:,5]  # Nodal susceptance
p_d = bus_data[:,2]/baseMVA  # Demand for active power at each node
q_d = bus_data[:,3]/baseMVA  # Demand for reactive power at each node

V_min = bus_data[:,12]**2  # Minimum voltage squared at each node
V_max = bus_data[:,11]**2  # Maximum voltage squared at each node

p_n_min = bus_data[:,21]/baseMVA  # Minimum active power at each node
p_n_max = bus_data[:,20]/baseMVA  # Maximum active power at each node
q_n_min = bus_data[:,16]/baseMVA  # Minimum reactive power at each node
q_n_max = bus_data[:,15]/baseMVA  # Maximum reactive power at each node

# theta_n min and max
theta_n_min = -np.pi/2*np.ones(len(bus_data))  # Minimum bus angle 
theta_n_max = np.pi/2*np.ones(len(bus_data))  # Maximum bus angle

# Define R_l, X_l and B_l (line resistances, reactances and susceptance)
R_l = line_data[:,2]  # Resistance of each line
X_l = line_data[:,3]  # Reactance of each line
B_l = line_data[:,4]  # Susceptance of each line

# theta_l min and max
theta_l_min = -np.pi/2*np.ones(len(line_data))  # Minimum line angle (from relaxation assumption)
theta_l_max = np.pi/2*np.ones(len(line_data))  # Maximum line angle (from relaxation assumption)

# Kl, the equivalent ampacity for line losses
K_l = 30 * np.ones(len(line_data))  #From czech assumption graph  # Ampacity for each line

In [15]:
# Variables
p_n = cp.Variable(num_nodes)  # Active power at node n
q_n = cp.Variable(num_nodes)  # Reactive power at node n
V_n = cp.Variable(num_nodes)  # Voltage magnitude squared at node n
theta_n = cp.Variable(num_nodes)  # Voltage angles at node n

p_sl = cp.Variable(num_lines)  # Active power at sending end of line l
q_sl = cp.Variable(num_lines)  # Reactive power at sending end of line l
p_ol = cp.Variable(num_lines)  # Active power losses on line l
q_ol = cp.Variable(num_lines)  # Reactive power losses on line l
K_ol = cp.Variable(num_lines)  # Branch Equivalent ampacity constraint on line l
theta_l = cp.Variable(num_lines)  # Voltage angles at line l


In [16]:
constraints = []

#Variables related to buses bounds 
for n in range(num_nodes): 
    # Voltage Magnitude bounds (1k):
    constraints.append(V_n[n] >= V_min[n])
    constraints.append(V_n[n] <= V_max[n])

    # Node angle bounds (1m):
    constraints.append(theta_n[n] >= theta_n_min[n])
    constraints.append(theta_n[n] <= theta_n_max[n])

    # Active power bounds (1n)
    constraints.append(p_n[n] >= p_n_min[n])
    constraints.append(p_n[n] <= p_n_max[n])

    # Reactive power bounds (1o)
    constraints.append(q_n[n] >= q_n_min[n])
    constraints.append(q_n[n] <= q_n_max[n])

#Variables related to lines bounds 
for l in range(num_lines): 
    # Line angle bounds (1l):
    constraints.append(theta_l[l] >= theta_l_min[l])
    constraints.append(theta_l[l] <= theta_l_max[l])

In [17]:
# Active power balance (1b)
for n in range(num_nodes):
    constraints.append(p_n[n] - p_d[n] == A_plus[n, :]@p_sl - A_minus[n, :]@p_ol + G_n[n]*V_n[n])

# Reactive power balance (1c)
for n in range(num_nodes):
    constraints.append(q_n[n] - q_d[n] == A_plus[n, :]@q_sl - A_minus[n, :]@q_ol - B_n[n]*V_n[n])
    
# Voltage drop constraint (1d):
for l in range(num_lines):
    constraints.append(V_n[sending_node[l]-1] - V_n[receiving_node[l]-1] == 2*R_l[l]*p_sl[l] + 2*X_l[l]*q_sl[l] - R_l[l]*p_ol[l] - X_l[l]*q_ol[l])


In [18]:
z_l = cp.Variable(num_lines)    # z_l >= sqrt(p_sl^2 + q_sl^2)

#qV_l = cp.Variable(num_lines)   # Relaxed term of q_ol[l] * V_n[sending_node[l]-1]
#for l in range(num_lines):
    # Upper and lower bounds for the product of q_ol and V_n
    #constraints.append(qV_l[l] >= q_ol[l] * V_min[sending_node[l]-1])  # Lower bound
    #constraints.append(qV_l[l] <= q_ol[l] * V_max[sending_node[l]-1])  # Upper bound

for l in range(num_lines):
    # Active and reactive power losses constraint (relaxed) (2b)
    constraints.append(K_ol[l] == (K_l[l] - V_n[sending_node[l]-1]*B_l[l]**2 + 2*q_sl[l]*B_l[l])*X_l[l])
    constraints.append(K_ol[l] >= q_ol[l])
    constraints.append(cp.SOC(z_l[l], np.sqrt(X_l[l])*cp.hstack([p_sl[l], q_sl[l]])))
    constraints.append(cp.norm(cp.vstack([2*z_l[l], q_ol[l] - V_n[sending_node[l]-1]]),2) <= q_ol[l] + V_n[sending_node[l]-1])
    #constraints.append(qV_l[l] >= z_l[l]**2 * X_l[l]) #the sending node should be a constant and not a variable
    
    # Power loss constraint (2c)
    constraints.append(p_ol[l] * X_l[l] == q_ol[l] * R_l[l])

    # Line angle constraint (1h)
    constraints.append(theta_l[l] == theta_n[sending_node[l]-1] - theta_n[receiving_node[l]-1])

In [19]:
for l in range(num_lines):
    # Linearized angle constraint (2d):
    constraints.append(theta_l[l] == X_l[l]*p_sl[l] - R_l[l]*q_sl[l])

In [20]:
for l in range(num_lines):
    # Equation (4g) :
    constraints.append(V_n[sending_node[l]-1] + V_n[receiving_node[l]-1] >= cp.norm(cp.vstack([2*theta_l[l]/np.sin(theta_l_max[l]), V_n[sending_node[l]-1] - V_n[receiving_node[l]-1]]),2))

In [21]:
# Objective: Minimize total generation cost
objective = 0
for i in range(num_nodes):
    a = bus_data[i, 25]  # Quadratic cost coefficient
    b = bus_data[i, 26]  # Linear cost coefficient
    c = bus_data[i, 27]  # Constant cost coefficient
    objective += a*cp.square(p_n[i]*baseMVA) + b*p_n[i]*baseMVA + c

# Defining the optimization problem
problem = cp.Problem(cp.Minimize(objective), constraints)

In [22]:
# Solve the problem
problem.solve(solver=cp.SCS)  # You can also use other solvers like 'ECOS' or 'MOSEK'

5310.082251206962

In [23]:
print("Problem Status:", problem.status)
print("Optimal Value of the Objective Function:", problem.value)

Problem Status: optimal
Optimal Value of the Objective Function: 5310.082251206962


In [13]:
"""eq2b_right = []
qolVsl = []
KolVsl = []
for l in range (num_lines):
    eq2b_right.append((z_l[l].value**2 * X_l[l])/V_n[sending_node[l]-1].value)
    qolVsl.append(q_ol[l].value * V_n[sending_node[l]-1].value)
    KolVsl.append(K_ol[l].value * V_n[sending_node[l]-1].value)"""

'eq2b_right = []\nqolVsl = []\nKolVsl = []\nfor l in range (num_lines):\n    eq2b_right.append((z_l[l].value**2 * X_l[l])/V_n[sending_node[l]-1].value)\n    qolVsl.append(q_ol[l].value * V_n[sending_node[l]-1].value)\n    KolVsl.append(K_ol[l].value * V_n[sending_node[l]-1].value)'

In [14]:
# Print with V_sl on the right side
print("Optimal Values of p_ol:", p_ol.value)
print("Optimal Values of q_ol:", q_ol.value)
print("Optimal Values of thata_l:", theta_l.value)
print("Optimal Values of thata_n:", theta_n.value)

Optimal Values of p_ol: [0.    0.003 0.011 0.    0.002 0.003 0.    0.016 0.004]
Optimal Values of q_ol: [0.061 0.014 0.049 0.051 0.016 0.026 0.106 0.079 0.035]
Optimal Values of thata_l: [ 0.052  0.029 -0.09   0.055  0.036 -0.043 -0.084  0.11  -0.042]
Optimal Values of thata_n: [ 0.007  0.107  0.071 -0.045 -0.073  0.016 -0.02   0.023 -0.087]


In [15]:
# Print with V_sl on the left side
"""print("Optimal Values of K_ol:", np.round(KolVsl,3))
print("Optimal Values of q_ol:", np.round(qolVsl,3))
print("Optimal Values of q_Vl:", qV_l.value)
print("Optimal Values of mult:", z_l.value**2 * X_l)"""

'print("Optimal Values of K_ol:", np.round(KolVsl,3))\nprint("Optimal Values of q_ol:", np.round(qolVsl,3))\nprint("Optimal Values of q_Vl:", qV_l.value)\nprint("Optimal Values of mult:", z_l.value**2 * X_l)'

In [16]:
print("Optimal valu for p_sl:", p_sl.value)
print("Optimal valu for q_sl:", q_sl.value)

Optimal valu for p_sl: [ 0.901  0.354 -0.548  0.943  0.384 -0.618 -1.344  0.723 -0.543]
Optimal valu for q_sl: [ 0.692  0.219 -0.095  0.397  0.202 -0.164 -0.393  0.202 -0.376]


In [17]:
print("Optimal valu for p_n:", p_n.value)
print("Optimal valu for q_n:", q_n.value)

Optimal valu for p_n: [ 0.901  1.344  0.943 -0.    -0.    -0.    -0.    -0.    -0.   ]
Optimal valu for q_n: [0.692 0.499 0.397 0.    0.    0.    0.    0.    0.   ]


In [18]:
print("Optimal valu for V_n:", V_n.value)

Optimal valu for V_n: [1.21  1.21  1.21  1.134 1.083 1.167 1.118 1.154 1.056]


In [19]:
p_d

array([0.  , 0.  , 0.  , 0.  , 0.9 , 0.  , 1.  , 0.  , 1.25])

In [66]:
sum(p_n.value)

3.188540894253992